In [1]:
print("연습문제 2-2. 영화 리뷰 감정 분석기")

연습문제 2-2. 영화 리뷰 감정 분석기


In [5]:
from dotenv import load_dotenv
import os
from enum import Enum
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers import EnumOutputParser, OutputFixingParser

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

class Sentiment(Enum):
    긍정 = "긍정"
    부정 = "부정"
    보통 = "보통"

enum_parser = EnumOutputParser(enum=Sentiment)

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

fixing_parser = OutputFixingParser.from_llm(parser=enum_parser, llm=llm)

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 영화 리뷰 감정 분석가다. "
               "리뷰를 읽고 감정을 '긍정', '부정', '보통' 중 하나로만 출력하라."),
    ("human", "{review}")
])

chain = prompt | llm | fixing_parser

# 테스트 리뷰들들
test_reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가..할말하않",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
]

for review in test_reviews:
    sentiment = chain.invoke({"review": review})
    print(f"리뷰: {review}\n | 감정 분류: {sentiment.value}\n")


리뷰: 이 영화 정말 재미없어요. 시간 낭비였습니다.
 | 감정 분류: 부정

리뷰: 배우들의 연기가..할말하않
 | 감정 분류: 부정

리뷰: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
 | 감정 분류: 보통

